In [1]:
import math

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import nestle
from os.path import join
import math
import pickle
from utilities import loadData, plot_ellipsoid_3d, lowest_point_z, lowest_point_y, lowest_point_x, SAVE_PATH, performRDP
%matplotlib widget

dataPath = r"C:\Users\ashamail\Box\Shared Materials_Abdullah_Hasan\Abdullah Data\allAtomsWithRings.csv"
savePath = r"C:\Users\ashamail\Desktop\drug-polymers\adbis extension\data\ellipsoids"
savepathDrive = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\adbis extension data"
oxygenPath = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Text Data Files\flavanone_R1_os_0-200K.csv"



In [2]:
data = pd.read_csv(dataPath)
oxygen = pd.read_csv(oxygenPath, header=None)
oxygen = oxygen.rename(columns={0: "x", 1: "y", 2: "z"})
df = pd.read_csv(join(savepathDrive, "min_z_points.csv"))

rfl1path = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Text Data Files\RFL1 Trajectory\rfl1_200k_0.txt"
PSP = 44
rfl1 = loadData(rfl1path)

In [3]:
def plot_ell_and_plane_2(ell, points):
    fig = plt.figure(figsize=(6., 6.))
    ax = fig.add_subplot(111, projection='3d')

    # z plane equations to plot

    x = np.linspace(min(points[0]),max(points[0]),100)
    y = np.linspace(min(points[1]),max(points[1]),100)
    z = np.linspace(min(points[2]),max(points[2]),100)
    a,b,c = 0,0,1
    z_min = lowest_point_z(ell.a, ell.ctr)
    x_min = lowest_point_x(ell.a, ell.ctr)
    y_min = lowest_point_y(ell.a, ell.ctr)

    X_1,Y_1 = np.meshgrid(x,y)
    Y_2,Z_1 = np.meshgrid(y,z)
    X_2,Z_2 = np.meshgrid(x,z)
    
    Z_PSP = (PSP - a*X_1 - b*Y_1) / c
    X_MIN = (x_min - a*Y_2 - b*Z_1) / c
    Y_MIN = (y_min - a*X_2 - b*Z_2) / c
    Z_MIN = (z_min - a*X_1 - b*Y_1) / c

    # ax.plot(x_min, y_min, z_min, marker="o", color="red")


    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='k', marker='.')
    ax.plot_surface(X_1, Y_1, Z_PSP)
    ax.plot_surface(X_1, Y_1, Z_MIN)
    # ax.plot_surface(X_MIN, Y_2, Z_1)
    # ax.plot_surface(X_2, Y_MIN, Z_2)
    plot_ellipsoid_3d(ell, ax)

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

def plot_ell_and_plane(ell, points):

    # Define the two plane equations
    z1 = PSP
    z2 = lowest_point_z(ell.a, ell.ctr)

    # Create an array of x and y values for plotting
    x = np.linspace(min(points[0]),max(points[0]), 100)
    y = np.linspace(min(points[1]),max(points[1]), 100)

    # Generate meshgrid from x and y
    X, Y = np.meshgrid(x, y)

    # Define Z for each plane
    Z1 = np.ones_like(X) * z1
    Z2 = np.ones_like(X) * z2

    # Define line points
    z = np.linspace(z1, z2, 100)
    x = np.ones_like(z)*45
    y = np.ones_like(z)*45

    # Plot the figure
    fig = plt.figure()
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z1, color='red', alpha=0.5)
    ax.plot_surface(X, Y, Z2, color='blue', alpha=0.5)
    ax.plot(x, y, z, color='black', linewidth=2)
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='k', marker='.')
    plot_ellipsoid_3d(ell, ax)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xticks([])

    plt.show()


def calc_distance_between_planes(ell, PSP):
    z_min = lowest_point_z(ell.a, ell.ctr)
    # print(z_min, PSP)
    return abs(z_min - PSP)

def saveEllCenters(data, path):
    centers = []
    for i in range(0, 200000):
        points = data[i]
        ell = nestle.bounding_ellipsoid(points)
        centers.append(ell.ctr)
    centers = np.array(centers)
    with open(path, "wb") as f:
        np.save(f, centers)
    f.close()
    return centers

def saveMinPointsZ(data, path):
    arr = []
    for i in range(0, 200000):
        points = data[i]
        ell = nestle.bounding_ellipsoid(points)
        z_min = lowest_point_z(ell.a, ell.ctr)
        arr.append([i, z_min])
        # print(z_min, i)
    df = pd.DataFrame(arr, columns =['timestep', 'min_z'])
    df.to_csv(path, index=False)
    return df

def runQueryDTheta(timesteps, oxygen):
    temp = pd.DataFrame(oxygen["z"][timesteps])
    temp["difference"] = temp["z"].apply(lambda x: abs(x - PSP))
    # display(temp[temp["difference"] < 3.5])
    return temp[temp["difference"] <= 3.5].index.values



In [6]:
# calculate all minimum z points and plot them vs time in next cell
min_z_points = []
for i in range(1):
    points = rfl1[i]
    ell = nestle.bounding_ellipsoid(points)
    
    plot_ell_and_plane(ell, points)
    # min_z_points.append(lowest_point_z(ell.a, ell.ctr))
    # print(ell.a, ell.ctr)

NameError: name 'ax' is not defined

In [36]:
plt.savefig(join(savePath, "ellipsoid min z to psp distance.pdf"))

In [148]:
# plt.clf()
# plt_1 = plt.figure(figsize=(15, 5))
# plt.plot(np.arange(0, 200000), min_z_points)
# plt.ylabel("min z coordinates")
# plt.xlabel("time")
# plt.show()

In [30]:
# save all centers of ellipsoids
# centers = saveEllCenters(rfl1, join(SAVE_PATH, "centers.txt"))
# with open(join(SAVE_PATH, "centers.txt"), 'rb') as f:
    # centers = np.load(f)
    # print(a[:10])

,0,1,2
0,16.8982,21.7579,69.6761
1,17.4634,21.7329,69.3243
2,17.5462,22.6110,69.3149
3,17.3647,22.6614,68.7115
4,17.7786,22.3793,69.6186
...,...,...,...
199995,52.6511,62.8906,48.3386
199996,52.6382,63.0113,48.5268
199997,52.2599,63.2609,48.3279
199998,52.0997,64.0660,47.8470


In [158]:
# for all timesteps
# can also plot an instance using the line below
# plot_ell_and_plane(ell, points)
d_deltas = [1,2,3,4]
countPerDelta = {}
for delta in d_deltas:
    count = 0
    df["difference"] = df["min_z"].apply(lambda x: abs(x - PSP))
    points_less_than_delta = df[df["difference"] < delta] #calculate ellipsoid and then calculate distance between min point and PSP using d_delte (1,2,3,4)
    # print(points_less_than_delta)
    timesteps = runQueryDTheta(np.array(points_less_than_delta["timestep"]), oxygen) #for each d_delta instances, run the d_theta query
    countPerDelta[delta] = timesteps

for key, val in countPerDelta.items():
    print(key, val.shape[0])

1 1383
2 3208
3 5801
4 9117


In [6]:
# #perform spm3d on the centers
# rfl1centersSPM = spm3d(centers, 0.5)
# print(rfl1centersSPM.shape)
# print(rfl1centersSPM)

(112716, 4)
[[1.65232814e+01 2.23942018e+01 6.98572603e+01 0.00000000e+00]
 [1.72788991e+01 2.28284266e+01 7.00962517e+01 4.00000000e+00]
 [1.68133426e+01 2.26674728e+01 6.92919164e+01 6.00000000e+00]
 ...
 [5.29564437e+01 6.31213299e+01 4.79556922e+01 1.99994000e+05]
 [5.18027979e+01 6.42039316e+01 4.82512107e+01 1.99997000e+05]
 [5.18213194e+01 6.41279507e+01 4.89323295e+01 1.99999000e+05]]


In [5]:
# #perform rdp on the centers
# rfl1centersRDP = performRDP(centers, 0.5)
# print(rfl1centersRDP.shape)
# print(rfl1centersRDP)


(114173, 4)
[[1.65232814e+01 2.23942018e+01 6.98572603e+01 0.00000000e+00]
 [1.68604005e+01 2.29457395e+01 6.93292278e+01 3.00000000e+00]
 [1.72788991e+01 2.28284266e+01 7.00962517e+01 4.00000000e+00]
 ...
 [5.29564437e+01 6.31213299e+01 4.79556922e+01 1.99994000e+05]
 [5.15989190e+01 6.46917170e+01 4.79105406e+01 1.99998000e+05]
 [5.18213194e+01 6.41279507e+01 4.89323295e+01 1.99999000e+05]]


In [7]:
# #save spm and rdp centers

# with open(join(SAVE_PATH, "rdpCenters0.5.txt"), "wb") as f:
#     np.save(f, rfl1centersRDP)
# f.close()

# with open(join(SAVE_PATH, "spmCenters0.5.txt"), "wb") as f:
#     np.save(f, rfl1centersSPM)
# f.close()
 